In [ ]:
def diagonalize(ionObj, ion, Field): 
    # assumes no ab anisotropy
    JdotB = muB*(Field[0]*cef.Operator.Jx(ionObj.J) + Field[1]*cef.Operator.Jy(ionObj.J) + Field[2]*cef.Operator.Jz(ionObj.J))*cef.LandeGFactor(ion)
    #we've defined this as a CFLevels object
    H = np.sum([a*b for a,b in zip(ionObj.O, ionObj.B)], axis=0)
    ionObj.diagonalize(H + JdotB.O) # this is just H = Hcef + Hmag
    return ionObj.eigenvalues 

def calcuateZeemanStuff(field, wavenum, B20, B40, B43, B60, B63, B66): 
    amp = [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1,]#[0, .15, .15, .2, 0.15,0.15,0.15,0.15,0.07,0.07, .1,.1,.1,.1,.1]
    Bparams =  {'B20': B20, 'B40':B40,'B43': B43, 'B60': B60, 'B63':B63,'B66':B66}
    ionObj = cef.CFLevels.Bdict(ion,Bparams)
    # this assumes identical fields placed perfectly along each axis
    evalsA, evalsB, evalsC = diagonalize(ionObj, ion, field)
    evals = [evalsA, evalsB, evalsC]
    for axis in [0,1,2]
        dE =[eval for eval in evals[axis]] # this is the spitting if everything is in the GS -> not necessarily true for finite temp
        # first, lets calculate the partition function - the microstates of the system don't change, jsut the number of abs line
        # we can make between them, so we can make this now
        # Z = sum_i (e^(-beta*E_i))
        Z = [np.exp(-Ei/kBT) for Ei in dE]
        Z = sum(Z)
        # now that we have the partition fn, we can calculate probabilities
        p = [1/Z*np.exp(-Ei/kBT) for Ei in dE]
        # okay, so now we want to add lines between non ground states 
        # we want the amplitude to be the probability -> main lines are already determined
        numLines = len(dE)
        for i in range(1,numLines): 
            # skip GS - already have those dE
            for j in range(i+1, numLines):
                temp = dE[j]-dE[i]
                dE.append(temp)
                amp.append(p[i]*amp[i])
        # so now we want to create a multi gaussian with centers at the eigenvalues
        wid = .5 # pulled this out my ass, .5cm-1 resolution roughly
        a = amp[0] #+ m[0]*field
        centers =dE 
        fun = gaussian(wavenum, a, centers[0]*meVToCm, wid)
        for i in range(len(centers[1:])):
            a = amp[i]# +m[i]*field
            fun += gaussian(wavenum, a, centers[i]*meVToCm, wid)    
    return funA, funB, funC




def spectrum(field, wavenum, B20, B40, B43, B60, B63, B66):     
    # assuming only H||B rn
    # assuming that x is an array
    try: # for the case we pass in a single B val
        fun = calcuateZeemanStuff(field, wavenum, B20, B40, B43, B60, B63, B66)
    except AttributeError : # for when we pass in an array 
        # pyCrystalField has trouble with the operators because its not *just* a function
        # so this error handling lets us have a function that works for single values and array inputs
        fun = []
        for b in field: 
            evals = diagonalizeC(ionObj, ion, b)
            dE =[eval for eval in evals] # this is the spitting if everything is in the GS -> not necessarily true for finite temp
            # first, lets calculate the partition function - the microstates of the system don't change, jsut the number of abs line
            # we can make between them, so we can make this now
            # Z = sum_i (e^(-beta*E_i))
            Z = [np.exp(-Ei/kBT) for Ei in dE]
            Z = sum(Z)
            # now that we have the partition fn, we can calculate probabilities
            p = [1/Z*np.exp(-Ei/kBT) for Ei in dE]
            # okay, so now we want to add lines between non ground states 
            # we want the amplitude to be the probability -> main lines are already determined
            numLines = len(dE)
            for i in range(1,numLines): 
                # skip GS - already have those dE
                for j in range(i+1, numLines):
                    temp = dE[j]-dE[i]
                    dE.append(temp)
                    amp.append(p[i])
            # so now we want to create a multi gaussian with centers at the eigenvalues
            wid = .3 # pulled this out my ass, .5cm-1 resolution roughly
            a = amp[0] #+ m[0]*field
            centers =dE 
            tempfun = gaussian(wavenum, a, centers[0]*meVToCm, wid)
            for i in range(len(centers[1:])):
                a = amp[i]# +m[i]*field
                tempfun += gaussian(wavenum, a, centers[i]*meVToCm, wid)
            fun.append(tempfun)
    return fun